# Podstawowy Przykład AutoGen

W tym przykładzie kodu użyjesz [AutoGen](https://aka.ms/ai-agents/autogen), frameworka AI, aby stworzyć podstawowego agenta.

Celem tego przykładu jest pokazanie kroków, które później wykorzystamy w dodatkowych przykładach kodu podczas implementacji różnych wzorców agentowych.


## Importuj potrzebne pakiety Pythona


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Utwórz klienta

W tym przykładzie użyjemy [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) do uzyskania dostępu do LLM.

`model` jest zdefiniowany jako `gpt-4o-mini`. Spróbuj zmienić model na inny dostępny w marketplace GitHub Models, aby zobaczyć różne wyniki.

Na szybki test uruchomimy prostą komendę - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiowanie Agenta

Teraz, gdy skonfigurowaliśmy `client` i potwierdziliśmy, że działa poprawnie, stwórzmy `AssistantAgent`. Każdemu agentowi można przypisać:  
**name** - Krótką nazwę, która będzie przydatna do odwoływania się do niego w przepływach z wieloma agentami.  
**model_client** - Klienta, którego utworzyłeś w poprzednim kroku.  
**tools** - Dostępne narzędzia, które Agent może wykorzystać do wykonania zadania.  
**system_message** - Metaprompt definiujący zadanie, zachowanie i ton LLM.  

Możesz zmienić wiadomość systemową, aby zobaczyć, jak LLM zareaguje. Omówimy `tools` w Lekcji #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Uruchom Agenta

Poniższa funkcja uruchomi agenta. Używamy metody `on_message`, aby zaktualizować stan Agenta o nową wiadomość.

W tym przypadku aktualizujemy stan o nową wiadomość od użytkownika, która brzmi: `"Zaplanuj mi świetne, słoneczne wakacje"`.

Możesz zmienić treść wiadomości, aby zobaczyć, jak LLM odpowiada w różny sposób.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić dokładność, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
